In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [6]:
from gensim.models import Word2Vec
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Giriş metni
query = "resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use"
query_tokens = query.split()

# Modeli yükle
model = Word2Vec.load("models0/lemmatized_cbow_win2_dim100.model")

# Lemmatized yorumları yükle
df = pd.read_csv("reviews_lemmatized.csv")
corpus = df["lemmatized"].fillna("").tolist()
corpus_tokens = [c.split() for c in corpus]

# Ortalama vektör hesaplayan fonksiyon
def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Giriş metni vektörü
query_vec = get_avg_vector(query_tokens, model)

# Tüm yorumlar için vektörler
doc_vectors = np.array([get_avg_vector(tokens, model) for tokens in corpus_tokens])

# Cosine similarity
similarities = cosine_similarity([query_vec], doc_vectors)[0]

# En benzer 5 yorumu yazdır
top_5_idx = similarities.argsort()[::-1][:5]
for i, idx in enumerate(top_5_idx):
    print(f"{i+1}. Skor: {similarities[idx]:.4f}")
    print(f"ID: {df.iloc[idx]['id']}")
    print(df.iloc[idx]["cleanedReview"])
    print("---")


1. Skor: 0.9403
ID: 36934
resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use
---
2. Skor: 0.9024
ID: 61313
latest update broke the app now randomly pauses and wont resume at that song rather goes back to another song thats already been played from that playlist numerous times
---
3. Skor: 0.8970
ID: 60915
ever since the recent updates the app has acting up the previous songs name and album art remains in the notifcation bar while the next song plays most of the times i cant even pause or change the song from the lock screen as i cant see anything weird please fix i am a paying customer and do not appreciate using such a pathethic app
---
4. Skor: 0.8933
ID: 35911
since the update the app crashes rando

In [11]:
import os
import pandas as pd
import numpy as np
from glob import glob
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Giriş metni
query = "resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use"
query_tokens = query.split()

# Anlamsal puan hesaplama
def otomatik_puan(sim):
    if sim >= 0.90:
        return 5
    elif sim >= 0.85:
        return 4
    elif sim >= 0.80:
        return 3
    elif sim >= 0.75:
        return 2
    else:
        return 1

# Ortalama vektör
def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Veri setlerini yükle
df_lemma = pd.read_csv("reviews_lemmatized.csv")
df_stem = pd.read_csv("reviews_stemmed.csv")

# Tüm model yollarını al
model_paths = sorted(glob("models0/*.model"))

# Her modeli sırayla işle
for model_path in model_paths:
    model_name = os.path.basename(model_path).replace(".model", "")
    print(f"\n🔍 Model: {model_name}")

    # Hangi veri seti kullanılacak?
    is_lemma = "lemmatized" in model_name
    df = df_lemma if is_lemma else df_stem
    corpus = df["lemmatized" if is_lemma else "stemmed"].fillna("").tolist()
    cleaned = df["cleanedReview"].fillna("").tolist()
    corpus_tokens = [text.split() for text in corpus]

    # Modeli yükle
    model = Word2Vec.load(model_path)
    query_vec = get_avg_vector(query_tokens, model)
    doc_vectors = np.array([get_avg_vector(tokens, model) for tokens in corpus_tokens])
    similarities = cosine_similarity([query_vec], doc_vectors)[0]

    # İlk 5 sonucu sırala
    top_5_idx = similarities.argsort()[::-1][:5]
    for i, idx in enumerate(top_5_idx):
        skor = similarities[idx]
        puan = otomatik_puan(skor)
        print(f"{i+1}. Skor: {skor:.4f} | Puan: {puan} | ID: {df.iloc[idx]['id']}")
        print(f"{df.iloc[idx]['cleanedReview']}")
        print("---")



🔍 Model: lemmatized_cbow_win2_dim100
1. Skor: 0.9403 | Puan: 5 | ID: 36934
resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use
---
2. Skor: 0.9024 | Puan: 5 | ID: 61313
latest update broke the app now randomly pauses and wont resume at that song rather goes back to another song thats already been played from that playlist numerous times
---
3. Skor: 0.8970 | Puan: 4 | ID: 60915
ever since the recent updates the app has acting up the previous songs name and album art remains in the notifcation bar while the next song plays most of the times i cant even pause or change the song from the lock screen as i cant see anything weird please fix i am a paying customer and do not appreciate using such a pathethi

1. Skor: 0.9846 | Puan: 5 | ID: 36934
resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use
---
2. Skor: 0.9638 | Puan: 5 | ID: 22139
dear spotify i understand that from time to time you need to refresh the looks of this app but since i am alive years i used some sort of machine or program to play music every single one of them has a the following order for the playback control buttons previous track playpause next track why on earth you think that giving me a huge playbutton on the left and putting the prevnext button next to each other and in the middle would improve my ux
---
3. Skor: 0.9635 | Puan: 5 | ID: 31628
given how much the subscription is the app is surprisingly littered with bugs and tidbits

1. Skor: 0.9008 | Puan: 5 | ID: 36934
resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use
---
2. Skor: 0.8616 | Puan: 4 | ID: 47729
if it aint broken dont fix it refusing automatic update is no longer an option stopped playing mid song and i came in to the app on a loading screen google play tells me i have the latest version even if as late as yesterday the update button was there for this app i didnt click it latest bug added cant search within a track while playing pausing resets playback time soundtrack has to be paused to change time in song
---
3. Skor: 0.8608 | Puan: 4 | ID: 38837
after the last update the app has been really buggy it often wont register when i go back on a song and it just keep

1. Skor: 0.9754 | Puan: 5 | ID: 36934
resent update has caused app to lose major functions listening to song and random stopspause track progresssound bar in app and notification dropdown not showing so unable to pause or next track unless tapping next song manually please patch asap long time user and this is first time its caused the app almost inoperable for continuing use
---
2. Skor: 0.9593 | Puan: 5 | ID: 35612
i just recently updated spotify and now its notifications is hard to remove unless i remove spotify from my task bar spotify keeps on cutting off the songs unexpectedly and this has been happening numerous times despite resetting my app at least whole times and also one more thing sometimes when i select a song notifications doesnt pop up below the screen to show you can stop skip or reverse the music and i had to keep resetting the app againx please fix these problems if you could
---
3. Skor: 0.9585 | Puan: 5 | ID: 30537
the most recent update as of made this app almost 